### Experiments for Section 4.2: Generalization

In [1]:
import numpy as np
import keras
import tensorflow as tf
import os
import json
import pickle
import random
from collections import namedtuple
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau

import models
import datasets
import entropy_estimators as ee

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## MLP

### 1. Baseline

In [3]:
cfg = {}
cfg['dataset'] = 'MNIST'
cfg['model'] = 'MLP'
cfg['width'] = [1024,1024,1024,1024]
cfg['noise_ratio'] = 0.0
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000

cfg['dropout'] = []
cfg['weight_decay'] = []
cfg['batch_norm'] = []
cfg['noise_ratio'] = 0

In [4]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/MLP_1024-1024-1024-1024_MNIST_SGD/run_1


In [5]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [6]:
trn, tst = datasets.get_dataset(cfg)

In [7]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              803840    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                10250     
Total params: 3,962,890
Trainable params: 3,962,890
Non-trainable params: 0
_________________________________________________

In [8]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=50)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

Epoch 1/1000
60000/60000 [==============================] - 8s 129us/step - loss: 0.2453 - accuracy: 0.9254
Training Accuracy: 0.971
Set best train accuracy to: 0.971
Epoch 2/1000
60000/60000 [==============================] - 7s 121us/step - loss: 0.0865 - accuracy: 0.9736
Training Accuracy: 0.985
Set best train accuracy to: 0.985
Epoch 3/1000
60000/60000 [==============================] - 7s 116us/step - loss: 0.0557 - accuracy: 0.9826
Training Accuracy: 0.988
Set best train accuracy to: 0.988
Epoch 4/1000
60000/60000 [==============================] - 7s 117us/step - loss: 0.0380 - accuracy: 0.9878
Training Accuracy: 0.991
Set best train accuracy to: 0.991
Epoch 5/1000
60000/60000 [==============================] - 7s 117us/step - loss: 0.0286 - accuracy: 0.9909
Training Accuracy: 0.995
Set best train accuracy to: 0.995
Epoch 6/1000
60000/60000 [==============================] - 7s 117us/step - loss: 0.0191 - accuracy: 0.9943
Training Accuracy: 0.996
Set best train accuracy to: 0.99

In [9]:
model.save(cfg['exp_name']+"/model_final")

In [10]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

Evaluating the model on the train and test set:
10000/10000 [==============================] - 1s 55us/step
Train loss = 0.002; Train accuracy = 1.000
Test loss = 0.075; Test accuracy = 0.984
Saving generalization/MLP_1024-1024-1024-1024_MNIST_SGD/run_1/metrics


In [11]:
smi_all = []

model = keras.models.load_model(cfg['exp_name']+"/model_final")

count = 0
for l, layer in enumerate(model.layers):
    if hasattr(layer, 'kernel'):
        count += 1
        int_model = keras.Model(inputs=model.inputs, outputs=model.layers[l].output)
        activity = int_model.predict(trn.X[:10000])
        smi = ee.compute_smi(x=activity, y=trn.y[:10000], m=1000)
        smi_all.append(smi)
        print(f'Layer {count}: SI(T;Y)={smi:.3f}')

fname = cfg['exp_name']+'/smi'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi_all, f, pickle.HIGHEST_PROTOCOL)

Layer 1: SI(T;Y)=0.282
Layer 2: SI(T;Y)=0.486
Layer 3: SI(T;Y)=0.849
Layer 4: SI(T;Y)=1.163
Layer 5: SI(T;Y)=1.480
Saving generalization/MLP_1024-1024-1024-1024_MNIST_SGD/run_1/smi


In [12]:
fname = cfg['exp_name']+'/smi_pen'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi_all[-2], f, pickle.HIGHEST_PROTOCOL)

Saving generalization/MLP_1024-1024-1024-1024_MNIST_SGD/run_1/smi_pen


### 2. Vary Dropout Probability

In [74]:
cfg = {}
cfg['dataset'] = 'MNIST'
cfg['model'] = 'MLP'
cfg['width'] = [1024,1024,1024,1024]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000

cfg['dropout'] = [0.2,0.2,0.2,0.2]
cfg['weight_decay'] = []
cfg['batch_norm'] = []
cfg['noise_ratio'] = 0

In [75]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+cfg['dataset']+'/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if len(cfg['dropout']) > 0:
    dropout =  '-'.join(map(str,cfg['dropout']))
    exp_name += '_Dropout_'+dropout
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/MLP_MNIST/MLP_1024-1024-1024-1024_MNIST_SGD_LabelNoise_0.01/run_1


In [76]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [77]:
trn, tst = datasets.get_dataset(cfg)

In [78]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1024)              803840    
_________________________________________________________________
dense_42 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_43 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_44 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_45 (Dense)             (None, 10)                10250     
Total params: 3,962,890
Trainable params: 3,962,890
Non-trainable params: 0
________________________________________________

In [79]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=20)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

model.save(cfg['exp_name']+"/model_final")

Epoch 1/1000
60000/60000 [==============================] - 8s 127us/step - loss: 0.3122 - accuracy: 0.9183
Training Accuracy: 0.966
Set best train accuracy to: 0.966
Epoch 2/1000
60000/60000 [==============================] - 8s 125us/step - loss: 0.1606 - accuracy: 0.9648
Training Accuracy: 0.973
Set best train accuracy to: 0.973
Epoch 3/1000
60000/60000 [==============================] - 7s 114us/step - loss: 0.1257 - accuracy: 0.9747
Training Accuracy: 0.980
Set best train accuracy to: 0.980
Epoch 4/1000
60000/60000 [==============================] - 7s 118us/step - loss: 0.1037 - accuracy: 0.9800
Training Accuracy: 0.984
Set best train accuracy to: 0.984
Epoch 5/1000
60000/60000 [==============================] - 7s 118us/step - loss: 0.0876 - accuracy: 0.9839
Training Accuracy: 0.982
Epoch 6/1000
60000/60000 [==============================] - 7s 114us/step - loss: 0.0778 - accuracy: 0.9850
Training Accuracy: 0.985
Set best train accuracy to: 0.985
Epoch 7/1000
60000/60000 [======

In [80]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

Evaluating the model on the train and test set:
10000/10000 [==============================] - 1s 61us/step
Train loss = 0.001; Train accuracy = 1.000
Test loss = 0.095; Test accuracy = 0.984
Saving generalization/MLP_MNIST/MLP_1024-1024-1024-1024_MNIST_SGD_LabelNoise_0.01/run_1/metrics


In [81]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")
int_model = keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
activity = int_model.predict(trn.X[:10000], batch_size=32)
smi = ee.compute_smi(x=activity, y=trn.y[:10000], m=1000)
print(f'SMI:{smi:.3f}')

fname = cfg['exp_name']+'/smi_pen'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi, f, pickle.HIGHEST_PROTOCOL)

SMI:0.637
Saving generalization/MLP_MNIST/MLP_1024-1024-1024-1024_MNIST_SGD_LabelNoise_0.01/run_1/smi_pen


### 3. Vary Label Noise Ratio

In [7]:
cfg = {}
cfg['dataset'] = 'MNIST'
cfg['model'] = 'MLP'
cfg['width'] = [1024,1024,1024,1024]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000

cfg['dropout'] = []
cfg['weight_decay'] = []
cfg['batch_norm'] = []
cfg['noise_ratio'] = 0.01

In [8]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+cfg['dataset']+'/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if cfg['noise_ratio'] > 0:
    exp_name += '_LabelNoise_'+str(cfg['noise_ratio'])
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/MLP_MNIST/MLP_1024-1024-1024-1024_MNIST_SGD_Dropout_0.2-0.2-0.2-0.2/run_1


In [9]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [10]:
trn, tst = datasets.get_dataset(cfg)

if cfg['noise_ratio'] > 0:
    with open(cfg['exp_name']+'/noisy_trn', 'wb') as f:
        pickle.dump(trn._asdict(), f, pickle.HIGHEST_PROTOCOL)
    with open(cfg['exp_name']+'/tst', 'wb') as f:
        pickle.dump(tst._asdict(), f, pickle.HIGHEST_PROTOCOL)

In [11]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              803840    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0   

In [ ]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=20)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

model.save(cfg['exp_name']+"/model_final")

Epoch 1/1000
60000/60000 [==============================] - 10s 172us/step - loss: 0.2881 - accuracy: 0.9108
Training Accuracy: 0.963
Set best train accuracy to: 0.963
Epoch 2/1000
60000/60000 [==============================] - 10s 168us/step - loss: 0.1172 - accuracy: 0.9639
Training Accuracy: 0.978
Set best train accuracy to: 0.978
Epoch 3/1000
60000/60000 [==============================] - 10s 169us/step - loss: 0.0844 - accuracy: 0.9733
Training Accuracy: 0.986
Set best train accuracy to: 0.986
Epoch 4/1000
60000/60000 [==============================] - 10s 167us/step - loss: 0.0660 - accuracy: 0.9793
Training Accuracy: 0.989
Set best train accuracy to: 0.989
Epoch 5/1000
60000/60000 [==============================] - 10s 173us/step - loss: 0.0515 - accuracy: 0.9836
Training Accuracy: 0.992
Set best train accuracy to: 0.992
Epoch 6/1000
60000/60000 [==============================] - 10s 170us/step - loss: 0.0451 - accuracy: 0.9860
Training Accuracy: 0.991
Epoch 7/1000
60000/60000 [

In [ ]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

if cfg['noise_ratio'] > 0:
    Dataset = namedtuple('Dataset',['X','Y','y'])
    with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
        trn = Dataset(**pickle.load(f))
    with open(cfg['exp_name']+'/tst', 'rb') as f:
        tst = Dataset(**pickle.load(f))

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
if cfg['noise_ratio'] > 0:
    Dataset = namedtuple('Dataset',['X','Y','y'])
    with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
        trn = Dataset(**pickle.load(f))
        
noisy_label = trn.Y.argmax(1)

model = keras.models.load_model(cfg['exp_name']+"/model_final")
int_model = keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
activity = int_model.predict(trn.X[:10000], batch_size=32)
smi = ee.compute_smi(x=activity, y=noisy_label[:10000], m=1000)
print(f'SMI:{smi:.3f}')

fname = cfg['exp_name']+'/smi_pen'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi, f, pickle.HIGHEST_PROTOCOL)

## CNN

### 1. Baseline

In [4]:
cfg = {}
cfg['dataset'] = 'Fashion_MNIST'
cfg['model'] = 'CNN_Global'
cfg['width'] = [512,512,512,512]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000
cfg['n_train'] = None

cfg['dropout'] = []
cfg['weight_decay'] = []
cfg['batch_norm'] = [] # set to [True,True,True,True] for use of batch norm in all layers
cfg['noise_ratio'] = 0

In [5]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+cfg['dataset']+'/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if cfg['n_train'] is not None:
    exp_name += '_'+str(cfg['n_train'])
if len(cfg['batch_norm']) > 0:
    bn =  '-'.join(map(str,cfg['batch_norm']))
    exp_name += '_BatchNorm_'+bn
if len(cfg['dropout']) > 0:
    dropout =  '-'.join(map(str,cfg['dropout']))
    exp_name += '_Dropout_'+dropout
if cfg['noise_ratio'] > 0:
    exp_name += '_LabelNoise_'+str(cfg['noise_ratio'])
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True/run_1


In [6]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [7]:
trn, tst = datasets.get_dataset(cfg)

In [8]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 512)       5120      
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 512)       2048      
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 512)       2359808   
_________________________________________________________________
batch_normalization_2 (Batch (None, 11, 11, 512)       2048      
_________________________________________________________________
activation_2 (Activation)    (None, 11, 11, 512)       0   

In [9]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=50)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

model.save(cfg['exp_name']+"/model_final")

Epoch 1/1000
60000/60000 [==============================] - 30s 503us/step - loss: 0.4424 - accuracy: 0.8388
Training Accuracy: 0.805
Set best train accuracy to: 0.805
Epoch 2/1000
60000/60000 [==============================] - 27s 444us/step - loss: 0.2996 - accuracy: 0.8909
Training Accuracy: 0.908
Set best train accuracy to: 0.908
Epoch 3/1000
60000/60000 [==============================] - 27s 443us/step - loss: 0.2520 - accuracy: 0.9083
Training Accuracy: 0.925
Set best train accuracy to: 0.925
Epoch 4/1000
60000/60000 [==============================] - 27s 443us/step - loss: 0.2158 - accuracy: 0.9218
Training Accuracy: 0.934
Set best train accuracy to: 0.934
Epoch 5/1000
60000/60000 [==============================] - 27s 446us/step - loss: 0.1887 - accuracy: 0.9316
Training Accuracy: 0.946
Set best train accuracy to: 0.946
Epoch 6/1000
60000/60000 [==============================] - 27s 444us/step - loss: 0.1625 - accuracy: 0.9420
Training Accuracy: 0.944
Epoch 7/1000
60000/60000 [

In [10]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

Evaluating the model on the train and test set:
10000/10000 [==============================] - 2s 167us/step
Train loss = 0.003; Train accuracy = 1.000
Test loss = 0.313; Test accuracy = 0.923
Saving generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True/run_1/metrics


In [11]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")
int_model = keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
activity = int_model.predict(trn.X[:10000], batch_size=32)
if len(activity.shape) > 2:
    activity = activity.reshape(activity.shape[0],-1)
smi = ee.compute_smi(x=activity, y=trn.y[:10000], m=1000)
print(f'SMI:{smi:.3f}')

fname = cfg['exp_name']+'/smi'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi, f, pickle.HIGHEST_PROTOCOL)

SMI:0.198
Saving generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True/run_1/smi


In [ ]:
# smi_all = []

# model = keras.models.load_model(cfg['exp_name']+"/model_final")

# count = 0
# for l, layer in enumerate(model.layers):
#     if isinstance(layer, keras.layers.Activation) or isinstance(layer, keras.layers.GlobalAveragePooling2D):
#         count += 1
#         int_model = keras.Model(inputs=model.inputs, outputs=model.layers[l].output)
#         activity = int_model.predict(trn.X[:10000], batch_size=32)
#         if len(activity.shape) > 2:
#             activity = activity.reshape(activity.shape[0],-1)
#         smi = ee.compute_smi(x=activity, y=trn.y[:10000], m=1000)
#         smi_all.append(smi)
#         print(f'Layer {count}: SI(T;Y)={smi:.3f}')

# fname = cfg['exp_name']+'/smi'
# print("Saving", fname)
# with open(fname, 'wb') as f:
#     pickle.dump(smi_all, f, pickle.HIGHEST_PROTOCOL)

### 2. Vary Dropout Probability

In [70]:
cfg = {}
cfg['dataset'] = 'Fashion_MNIST'
cfg['model'] = 'CNN_Global'
cfg['width'] = [512,512,512,512]
cfg['noise_ratio'] = 0.0
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000
cfg['n_train'] = None

cfg['dropout'] = [0.4,0.4,0.4,0.4]
cfg['weight_decay'] = []
cfg['batch_norm'] = [True,True,True,True]
cfg['noise_ratio'] = 0

In [71]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+cfg['dataset']+'/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if len(cfg['batch_norm']) > 0:
    bn =  '-'.join(map(str,cfg['batch_norm']))
    exp_name += '_BatchNorm_'+bn
if len(cfg['dropout']) > 0:
    dropout =  '-'.join(map(str,cfg['dropout']))
    exp_name += '_Dropout_'+dropout
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True_Dropout_0.4-0.4-0.4-0.4/run_1


In [72]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [73]:
trn, tst = datasets.get_dataset(cfg)

In [74]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 13, 13, 512)       5120      
_________________________________________________________________
batch_normalization_13 (Batc (None, 13, 13, 512)       2048      
_________________________________________________________________
activation_33 (Activation)   (None, 13, 13, 512)       0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 11, 11, 512)       2359808   
_________________________________________________________________
batch_normalization_14 (Batc (None, 11, 11, 512)       204

In [75]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=50)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

model.save(cfg['exp_name']+"/model_final")

Epoch 1/1000
60000/60000 [==============================] - 28s 461us/step - loss: 0.5410 - accuracy: 0.7987
Training Accuracy: 0.858
Set best train accuracy to: 0.858
Epoch 2/1000
60000/60000 [==============================] - 28s 467us/step - loss: 0.3832 - accuracy: 0.8609
Training Accuracy: 0.877
Set best train accuracy to: 0.877
Epoch 3/1000
60000/60000 [==============================] - 30s 501us/step - loss: 0.3358 - accuracy: 0.8769
Training Accuracy: 0.894
Set best train accuracy to: 0.894
Epoch 4/1000
60000/60000 [==============================] - 32s 530us/step - loss: 0.3082 - accuracy: 0.8875
Training Accuracy: 0.901
Set best train accuracy to: 0.901
Epoch 5/1000
60000/60000 [==============================] - 32s 528us/step - loss: 0.2861 - accuracy: 0.8954
Training Accuracy: 0.893
Epoch 6/1000
60000/60000 [==============================] - 32s 527us/step - loss: 0.2741 - accuracy: 0.9003
Training Accuracy: 0.918
Set best train accuracy to: 0.918
Epoch 7/1000
60000/60000 [

In [76]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

Evaluating the model on the train and test set:
10000/10000 [==============================] - 2s 151us/step
Train loss = 0.006; Train accuracy = 1.000
Test loss = 0.273; Test accuracy = 0.931
Saving generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True_Dropout_0.4-0.4-0.4-0.4/run_1/metrics


In [77]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")
int_model = keras.Model(inputs=model.inputs, outputs=model.layers[-4].output)
activity = int_model.predict(trn.X[:10000], batch_size=32)
if len(activity.shape) > 2:
    activity = activity.reshape(activity.shape[0],-1)
smi = ee.compute_smi(x=activity, y=trn.y[:10000], m=1000)
print(f'SMI:{smi:.3f}')

fname = cfg['exp_name']+'/smi'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi, f, pickle.HIGHEST_PROTOCOL)

SMI:0.305
Saving generalization/CNN_Global_Fashion_MNIST/CNN_Global_512-512-512-512_Fashion_MNIST_SGD_BatchNorm_True-True-True-True_Dropout_0.4-0.4-0.4-0.4/run_1/smi


### 3. Vary Label Noise Ratio

In [74]:
cfg = {}
cfg['dataset'] = 'Fashion_MNIST'
cfg['model'] = 'CNN_Global'
cfg['width'] = [512,512,512,512]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 1000
cfg['n_train'] = None

cfg['dropout'] = []
cfg['weight_decay'] = []
cfg['batch_norm'] = [True,True,True,True]
cfg['noise_ratio'] = 1.0

In [75]:
run = 1
arch =  '-'.join(map(str,cfg['width']))
exp_name = 'generalization/'+cfg['model']+'_'+cfg['dataset']+'/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if len(cfg['dropout']) > 0:
    dropout =  '-'.join(map(str,cfg['dropout']))
    exp_name += '_Dropout_'+dropout
if len(cfg['batch_norm']) > 0:
    bn =  '-'.join(map(str,cfg['batch_norm']))
    exp_name += '_BatchNorm_'+bn
if cfg['noise_ratio'] > 0:
    exp_name += '_LabelNoise_'+str(cfg['noise_ratio'])
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

Making directory generalization/CNN_Global_512-512-512-512_CIFAR10_SGD_WD_0.0001-0.0001-0.0001-0.0001/run_1


In [76]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [77]:
trn, tst = datasets.get_dataset(cfg)

if cfg['noise_ratio'] > 0:
    with open(cfg['exp_name']+'/noisy_trn', 'wb') as f:
        pickle.dump(trn._asdict(), f, pickle.HIGHEST_PROTOCOL)
    with open(cfg['exp_name']+'/tst', 'wb') as f:
        pickle.dump(tst._asdict(), f, pickle.HIGHEST_PROTOCOL)

In [80]:
model = models.get_model(cfg, trn)
model.save(cfg['exp_name']+"/model_initial")
model.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 15, 15, 512)       14336     
_________________________________________________________________
activation_49 (Activation)   (None, 15, 15, 512)       0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 13, 13, 512)       2359808   
_________________________________________________________________
activation_50 (Activation)   (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 6, 6, 512)         2359808   
_________________________________________________________________
activation_51 (Activation)   (None, 6, 6, 512)         0  

In [81]:
class CustomEarlyStopping(keras.callbacks.Callback):
    def __init__(self, patience=0, threshold=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_trn_acc = 0
    
    def on_epoch_end(self, epoch, logs=None):
        trn_acc = self.model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)[1]       
        print('Training Accuracy: %0.03f'%trn_acc)
        if np.greater(np.round(trn_acc,3), np.round(self.best_trn_acc,3)):
            print('Set best train accuracy to: %0.03f'%trn_acc)
            self.best_trn_acc = trn_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

custom_ES = CustomEarlyStopping(patience=50)
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [custom_ES, lr_decay])

model.save(cfg['exp_name']+"/model_final")

Epoch 1/1000
50000/50000 [==============================] - 11s 226us/step - loss: 2.1999 - accuracy: 0.2347
Training Accuracy: 0.306
Set best train accuracy to: 0.306
Epoch 2/1000
50000/50000 [==============================] - 11s 220us/step - loss: 1.9173 - accuracy: 0.3540
Training Accuracy: 0.382
Set best train accuracy to: 0.382
Epoch 3/1000
50000/50000 [==============================] - 11s 219us/step - loss: 1.7530 - accuracy: 0.4172
Training Accuracy: 0.457
Set best train accuracy to: 0.457
Epoch 4/1000
50000/50000 [==============================] - 11s 219us/step - loss: 1.6307 - accuracy: 0.4641
Training Accuracy: 0.475
Set best train accuracy to: 0.475
Epoch 5/1000
50000/50000 [==============================] - 11s 214us/step - loss: 1.5257 - accuracy: 0.5068
Training Accuracy: 0.524
Set best train accuracy to: 0.524
Epoch 6/1000
50000/50000 [==============================] - 11s 214us/step - loss: 1.4408 - accuracy: 0.5391
Training Accuracy: 0.555
Set best train accuracy to

In [82]:
model = keras.models.load_model(cfg['exp_name']+"/model_final")

if cfg['noise_ratio'] > 0:
    Dataset = namedtuple('Dataset',['X','Y','y'])
    with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
        trn = Dataset(**pickle.load(f))
    with open(cfg['exp_name']+'/tst', 'rb') as f:
        tst = Dataset(**pickle.load(f))

print('Evaluating the model on the train and test set:')
trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=1)
train_loss = trn_results[0]
train_acc = trn_results[1]
tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=1)
test_loss = tst_results[0]
test_acc = tst_results[1]
print('Train loss = %0.03f; Train accuracy = %0.03f'%(train_loss, train_acc))
print('Test loss = %0.03f; Test accuracy = %0.03f'%(test_loss, test_acc))

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

Evaluating the model on the train and test set:
10000/10000 [==============================] - 2s 158us/step
Train loss = 0.195; Train accuracy = 1.000
Test loss = 1.429; Test accuracy = 0.750
Saving generalization/CNN_Global_512-512-512-512_CIFAR10_SGD_WD_0.0001-0.0001-0.0001-0.0001/run_1/metrics


In [83]:
if cfg['noise_ratio'] > 0:
    Dataset = namedtuple('Dataset',['X','Y','y'])
    with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
        trn = Dataset(**pickle.load(f))
        
noisy_label = trn.Y.argmax(1)

model = keras.models.load_model(cfg['exp_name']+"/model_final")
int_model = keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
activity = int_model.predict(trn.X[:10000], batch_size=32)
if len(activity.shape) > 2:
    activity = activity.reshape(activity.shape[0],-1)
smi = ee.compute_smi(x=activity, y=noisy_label[:10000], m=1000)
print(f'SMI:{smi:.3f}')

fname = cfg['exp_name']+'/smi'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi, f, pickle.HIGHEST_PROTOCOL)

SMI:0.034
Saving generalization/CNN_Global_512-512-512-512_CIFAR10_SGD_WD_0.0001-0.0001-0.0001-0.0001/run_1/smi
